In [1]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import time

In [2]:
# Define
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

def mediapipe_detection (image, model):    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      # Convert the image from BGR to RGB color space
    image.flags.writeable = False                       # Set the writeable flag of the image to False to prevent modification   
    results = model.process(image)                      # Process the image using the provided model (presumably the holistic model)   
    image.flags.writeable = True                        # Set the writeable flag of the image back to True for further processing   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)      # Convert the image back from RGB to BGR color space
    return image, results

def draw_styled_landmarks(image, results):
    # Draw left hand Connection
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color = (121, 22, 76), thickness = 2, circle_radius = 4),
                                mp_drawing.DrawingSpec(color = (121, 44, 250), thickness =2, circle_radius =2)
                                ) 
    
    # Draw right hand Connection   
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 1, circle_radius = 4),
                                mp_drawing.DrawingSpec(color = (245, 66, 230), thickness =1, circle_radius =2)
                                )   
    
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

# Daftar class actions
actions = np.array(['A', 'B', 'C', 'D', 'E', 
                    'F', 'G', 'H', 'I', 'J', 
                    'K', 'L', 'M', 'N', 'O', 
                    'P', 'Q', 'R', 'S', 'T', 
                    'U', 'V', 'W', 'X', 'Y', 
                    'Z', 'Halo', 'Perkenalkan', 
                    'Nama', 'Saya', 'Senang', 'Bertemu', 'Kamu' ]) 

In [3]:
ls

 Volume in drive D is Data
 Volume Serial Number is 300E-DC17

 Directory of d:\capstone_project_bangkit\machine-learning

15/06/2024  11:25    <DIR>          .
15/06/2024  10:35    <DIR>          ..
15/06/2024  11:25           314.455 dataset-create-kar.ipynb
15/06/2024  11:05    <DIR>          model
15/06/2024  11:19            17.481 testing-convLSTM-realTime.-dt-kar.ipynb
15/06/2024  11:19            17.481 testing-GRU-LSTM-realTime.-dt-kar copy.ipynb
15/06/2024  10:58    <DIR>          train-code
               3 File(s)        349.417 bytes
               4 Dir(s)   4.120.891.392 bytes free


In [6]:
model_GRU = tf.keras.models.load_model("./model/33C_GRU_acc098_loss01_seq25.h5")

In [3]:
model_LSTM = tf.keras.models.load_model("./model/33C_LSTM_acc098_loss01_seq25.h5")

**Testing GRU Model**

In [10]:
# Initialize variables

# Testing GRU Model
sequence = []
sentence = []
predictions = []

threshold = 0.95

state = 'countdown'
countdown_time = 3
display_time = 2
start_time = time.time()

cap = cv2.VideoCapture(0)
mp_holistic = mp.solutions.holistic

# Helper function to show countdown
def show_countdown(image, seconds_left):
    cv2.putText(image, f'Memulai prediksi dalam {seconds_left}..', (100, 250), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4, cv2.LINE_AA)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read frame
        ret, frame = cap.read()

        current_time = time.time()
        
        if state == 'countdown':
            seconds_left = countdown_time - int(current_time - start_time)
            image = frame.copy()
            show_countdown(image, seconds_left)
            
            if seconds_left <= 0:
                state = 'collecting'
                start_time = current_time
                sequence = []
        
        elif state == 'collecting':
            # Perform detection
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            
            # Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            
            if len(sequence) == 30:
                res = model_GRU.predict(np.expand_dims(sequence, axis=0))[0]

                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                
                # Update sentence based on prediction
                if np.unique(predictions[-28:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                
                if len(sentence) > 5:
                    sentence = sentence[-5:]

                state = 'displaying'
                start_time = current_time
        
        elif state == 'displaying':
            # Display the prediction
            if len(sentence) > 0:
                cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            if (current_time - start_time) > display_time:
                state = 'countdown'
                start_time = current_time

        # Show the frame
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 395ms/step
C
1/1 [==============================] - 0s 20ms/step
A
1/1 [==============================] - 0s 20ms/step
B
1/1 [==============================] - 0s 20ms/step
C
1/1 [==============================] - 0s 21ms/step
D
1/1 [==============================] - 0s 22ms/step
E
1/1 [==============================] - 0s 20ms/step
F
1/1 [==============================] - 0s 20ms/step
G
1/1 [==============================] - 0s 22ms/step
X
1/1 [==============================] - 0s 21ms/step
H
1/1 [==============================] - 0s 22ms/step
I
1/1 [==============================] - 0s 26ms/step
J
1/1 [==============================] - 0s 21ms/step
S
1/1 [==============================] - 0s 21ms/step
K
1/1 [==============================] - 0s 20ms/step
L
1/1 [==============================] - 0s 19ms/step
M
1/1 [==============================] - 0s 89ms/step
M
1/1 [==============================] - 0s 84ms/step
M
1/1 [======================

**Testing LSTM Model**

In [12]:
# Initialize variables
sequence = []
sentence = []
predictions = []

threshold = 0.95

state = 'countdown'
countdown_time = 3
display_time = 2
start_time = time.time()

cap = cv2.VideoCapture(0)
mp_holistic = mp.solutions.holistic

# Helper function to show countdown
def show_countdown(image, seconds_left):
    cv2.putText(image, f'Memulai prediksi dalam {seconds_left}..', (100, 250), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4, cv2.LINE_AA)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read frame
        ret, frame = cap.read()

        current_time = time.time()
        
        if state == 'countdown':
            seconds_left = countdown_time - int(current_time - start_time)
            image = frame.copy()
            show_countdown(image, seconds_left)
            
            if seconds_left <= 0:
                state = 'collecting'
                start_time = current_time
                sequence = []
        
        elif state == 'collecting':
            # Perform detection
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            
            # Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            
            if len(sequence) == 30:
                res = model_LSTM.predict(np.expand_dims(sequence, axis=0))[0]

                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                
                # Update sentence based on prediction
                if np.unique(predictions[-28:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                
                if len(sentence) > 5:
                    sentence = sentence[-5:]

                state = 'displaying'
                start_time = current_time
        
        elif state == 'displaying':
            # Display the prediction
            if len(sentence) > 0:
                cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            if (current_time - start_time) > display_time:
                state = 'countdown'
                start_time = current_time

        # Show the frame
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step
Z
1/1 [==============================] - 0s 27ms/step
A
1/1 [==============================] - 0s 24ms/step
B
1/1 [==============================] - 0s 22ms/step
Kamu
1/1 [==============================] - 0s 23ms/step
Kamu
1/1 [==============================] - 0s 24ms/step
Kamu
1/1 [==============================] - 0s 23ms/step
Kamu
1/1 [==============================] - 0s 23ms/step
Kamu
1/1 [==============================] - 0s 23ms/step
Kamu
1/1 [==============================] - 0s 102ms/step
D
1/1 [==============================] - 0s 100ms/step
E
1/1 [==============================] - 0s 107ms/step
F
1/1 [==============================] - 0s 113ms/step
G
1/1 [==============================] - 0s 97ms/step
H
1/1 [==============================] - 0s 98ms/step
I
1/1 [==============================] - 0s 26ms/step
Kamu
1/1 [==============================] - 0s 23ms/step
J
1/1 [==============================] - 0s 25ms/step
K
1/1